In [1]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


SEARCH_WORD = "EXA FIRST"

DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "アスカ狭山店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    }

# 検索キーワードよりホール名取得
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
        print(f"🔍 '{SEARCH_WORD}' を含むホール名が見つかりました。")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals):
    bb_medals = 239.25
    rb_medals = 95.25
    cherry_rate_high = 0.04228
    cherry_rate_low = 0.05847
    replay_rate = 0.411
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"]).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
model_list = df["model_name"].unique()
print(f'以下のモデルが含まれています')
for i, model in enumerate(model_list):
    print(f'{i}: {model}')

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 57760 x 16
以下のモデルが含まれています
0: ゴーゴージャグラー3
1: マイジャグラーV
2: ファンキージャグラー2
3: ミスタージャグラー
4: アイムジャグラーEX-TP
5: ハッピージャグラーVIII
6: ウルトラミラクルジャグラー
7: ジャグラーガールズ
8: SミスタージャグラーKK


In [2]:
df.head()

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,grape_rate,Total_rate,month,day,weekday,unit_last
0,EXA FIRST,2025-04-18,ゴーゴージャグラー3,1001,3212,9,9,-1124,356.9,356.9,6.35,178.4,2025-04,18,4,1
1,EXA FIRST,2025-04-18,ゴーゴージャグラー3,1002,2074,6,4,-968,345.7,518.5,6.64,207.4,2025-04,18,4,2
2,EXA FIRST,2025-04-18,ゴーゴージャグラー3,1003,910,2,2,-409,455.0,455.0,5.87,227.5,2025-04,18,4,3
3,EXA FIRST,2025-04-18,ゴーゴージャグラー3,1004,3883,16,15,-44,242.7,258.9,6.77,125.3,2025-04,18,4,4
4,EXA FIRST,2025-04-18,ゴーゴージャグラー3,1005,586,1,0,-556,586.0,0.0,6.72,586.0,2025-04,18,4,5


## RB_RATE

In [3]:
model_name = model_list[1]
today = datetime.date.today()
target_date = today - datetime.timedelta(days=1)
start = today - relativedelta(days=1)
end = today - relativedelta(days=61)
unit_no = 1021

df_tmp = df[(df["model_name"] == model_name)].copy()
df_tmp = df_tmp[(df_tmp["date"].dt.date <= start) & (df_tmp["date"].dt.date >= end)]
df_tmp = df_tmp[df_tmp["unit_no"] >= unit_no]

game = df_tmp.pivot_table(index=["model_name", "unit_no", ], columns="date", values="game", aggfunc="sum",)
bb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="BB", aggfunc="sum",)
rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="RB", aggfunc="sum",)
rb_rate = (game / rb).round(1)
total_rate = (game / (bb+rb)).round(1)

rb_rate[today] = ""
rb_rate = rb_rate.iloc[:, 7:].iloc[:, ::-1]

medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="medals", aggfunc="sum",)
rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
rolling_7d_sum.columns = [f"{col.strftime('%y-%m-%d')}_7d_sum" for col in rolling_7d_sum.columns]
rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]

C:\Users\mkyag\AppData\Local\Temp\ipykernel_17592\288722337.py:22: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]


In [4]:
print(rolling_7d_sum.shape)
print(rb_rate.shape)
display(rolling_7d_sum.head(3))
display(rb_rate.head(3))

(61, 53)
(61, 53)


25-04-12_7d_sum  25-04-11_7d_sum  25-04-10_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021             -1245.0          -2389.0          -1068.0   
           1022              2038.0           2574.0           2248.0   
           1023             -1273.0           -282.0           -988.0   

                    25-04-09_7d_sum  25-04-08_7d_sum  25-04-07_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021             -1333.0          -3765.0          -1129.0   
           1022              2004.0           1406.0           1583.0   
           1023              -200.0            944.0             38.0   

                    25-04-06_7d_sum  25-04-05_7d_sum  25-04-04_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021              -667.0             24.0           1016.0   
           1022               904.0            118.0            827.0   
           1023                 2.0            758.0           -598.0   

                    25-04-03_7d_sum  ...  25-02-26_7d_sum  25-02-25_7d_sum  \
model_name unit_no                   ...                                     
マイジャグラーV   1021               528.0  ...           4448.0           4468.0   
           1022               262.0  ...           2424.0            318.0   
           1023             -1548.0  ...              8.0           1623.0   

                    25-02-24_7d_sum  25-02-23_7d_sum  25-02-22_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021              3023.0           3576.0           -583.0   
           1022             -1285.0           -914.0           -179.0   
           1023              1929.0          -1601.0            308.0   

                    25-02-21_7d_sum  25-02-20_7d_sum  25-02-19_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021              -756.0          -2277.0          -2033.0   
           1022              1976.0            541.0           1235.0   
           1023                90.0           1075.0           1808.0   

                    25-02-18_7d_sum  25-02-17_7d_sum  
model_name unit_no                                    
マイジャグラーV   1021             -2383.0          -1550.0  
           1022              1985.0            870.0  
           1023               111.0           1091.0  

[3 rows x 53 columns]

date               2025-04-19  2025-04-18 00:00:00  2025-04-17 00:00:00  \
model_name unit_no                                                        
マイジャグラーV   1021                              269.1                459.3   
           1022                              638.0                484.6   
           1023                              423.7                338.9   

date                2025-04-16 00:00:00  2025-04-15 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   434.9                251.1   
           1022                   360.9                326.5   
           1023                  1230.0                239.3   

date                2025-04-14 00:00:00  2025-04-13 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   940.8                659.8   
           1022                   306.4                613.0   
           1023                   279.1                448.5   

date                2025-04-12 00:00:00  2025-04-11 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   375.3                314.1   
           1022                   271.6                372.2   
           1023                   329.8                565.1   

date                2025-04-10 00:00:00  ...  2025-03-07 00:00:00  \
model_name unit_no                       ...                        
マイジャグラーV   1021                   616.0  ...                307.8   
           1022                   314.5  ...                586.0   
           1023                   441.8  ...                274.5   

date                2025-03-06 00:00:00  2025-03-05 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   451.0                416.2   
           1022                   333.7                336.0   
           1023                   362.8                393.0   

date                2025-03-04 00:00:00  2025-03-03 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   324.2                220.7   
           1022                   627.0                279.3   
           1023                   286.5                445.7   

date                2025-03-02 00:00:00  2025-03-01 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   246.5                288.1   
           1022                   388.2                301.6   
           1023                   306.3                478.3   

date                2025-02-26 00:00:00  2025-02-25 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   549.0                274.9   
           1022                   342.1                487.6   
           1023                   475.0                300.3   

date                2025-02-24 00:00:00  
model_name unit_no                       
マイジャグラーV   1021                   445.5  
           1022                   429.7  
           1023                   349.9  

[3 rows x 53 columns]

In [7]:
# 同じ列数を前提（もしくは短い方に合わせる）
columns_interleaved = [
    col for pair in zip(rb_rate.columns, rolling_7d_sum.columns) for col in pair
]
merged = pd.concat([rb_rate, rolling_7d_sum], axis=1)[columns_interleaved]
# 空行を追加
empty_index = pd.MultiIndex.from_tuples([(model_name, "")], names=merged.index.names)
merged_with_blanks = pd.DataFrame(
    [[""] * merged.shape[1]], columns=merged.columns, index=empty_index
)
empty_row = merged_with_blanks.copy()
blocks = [
    merged.iloc[0:6],
    merged.iloc[6:12],
    merged.iloc[12:21],
    merged.iloc[21:30],
    merged.iloc[30:46],
    merged.iloc[46:54],
    merged.iloc[54:61],
]
for block in blocks:
    merged_with_blanks = pd.concat([merged_with_blanks, block, empty_row])
merged_with_blanks.head(60)

2025-04-19 25-04-12_7d_sum 2025-04-18 00:00:00  \
model_name unit_no                                                  
マイジャグラーV                                                            
           1021                       -1245.0               269.1   
           1022                        2038.0               638.0   
           1023                       -1273.0               423.7   
           1024                        -751.0               510.2   
           1025                        4811.0               352.3   
           1026                        1068.0               333.2   
                                                                    
           1027                        1239.0               435.3   
           1028                       -3946.0               320.6   
           1029                       -5886.0               305.8   
           1030                       -1271.0               307.0   
           1031                       -2028.0               267.4   
           1032                        7655.0               192.6   
                                                                    
           1033                        -869.0               428.0   
           1034                       -2447.0               581.6   
           1035                       -1708.0               326.8   
           1036                        5538.0                 inf   
           1037                        -896.0               303.3   
           1038                         723.0               387.6   
           1039                       -1758.0               348.1   
           1040                        1691.0               410.7   
           1041                       -4120.0               335.3   
                                                                    
           1042                       -2214.0               397.6   
           1043                       -3697.0               325.2   
           1044                        3068.0               647.7   
           1045                         400.0                 inf   
           1046                        -777.0               267.6   
           1047                       -2126.0               342.8   
           1048                       -3189.0               697.5   
           1049                       -1527.0               360.6   
           1050                        -103.0               568.4   
                                                                    
           1069                       -1604.0               332.1   
           1070                        1194.0               607.0   
           1071                         509.0               190.0   
           1072                       -5586.0               288.4   
           1073                       -3935.0               400.0   
           1074                        1682.0               292.9   
           1075                         744.0               325.6   
           1076                         253.0               404.8   
           1077                        3923.0               362.6   
           1078                        1173.0               254.4   
           1079                         915.0               241.1   
           1080                         402.0               329.6   
           1081                       -1006.0               297.7   
           1082                        3108.0               373.7   
           1083                        4437.0               267.1   
           1084                       -2258.0               238.6   
                                                                    
           1085                         142.0               416.9   
           1086                       -2562.0               447.7   
           1087                        -637.0               301.4   
           1088                       -6135.0               299.0   
           1089  

In [6]:
merged_with_blanks.to_csv(f"{model_name}_merged.csv", encoding="utf_8_sig")